## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country
0,0,Miandrivazo,-19.5167,45.4667,76.28,few clouds,56,17,8.48,MG
1,1,Barrow,71.2906,-156.7887,26.62,overcast clouds,93,90,0.00,US
2,2,Busselton,-33.6500,115.3333,50.09,clear sky,74,7,7.36,AU
3,3,Vila Franca Do Campo,37.7167,-25.4333,70.21,clear sky,62,8,5.82,PT
4,4,Kapaa,22.0752,-159.3190,75.52,light rain,93,40,9.22,US


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country
0,0,Miandrivazo,-19.5167,45.4667,76.28,few clouds,56,17,8.48,MG
4,4,Kapaa,22.0752,-159.3190,75.52,light rain,93,40,9.22,US
6,6,Roebourne,-20.7833,117.1333,81.52,few clouds,32,20,7.09,AU
10,10,Odweyne,9.4092,45.0640,76.24,scattered clouds,49,43,16.28,SO
13,13,Arraial Do Cabo,-22.9661,-42.0278,76.96,clear sky,60,0,16.11,BR
19,19,Kieta,-6.2167,155.6333,77.00,overcast clouds,86,100,4.90,PG
26,26,Cabo San Lucas,22.8909,-109.9124,82.51,clear sky,75,1,3.00,MX
30,30,Kirakira,-10.4544,161.9205,78.31,broken clouds,78,82,7.58,SB
38,38,Meulaboh,4.1363,96.1285,76.73,overcast clouds,89,100,5.88,ID
41,41,Saint-Philippe,-21.3585,55.7679,76.59,light rain,83,85,7.36,RE


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                200
City                   200
Lat                    200
Lng                    200
Max Temp               200
Current Description    200
Humidity               200
Cloudiness             200
Wind Speed             200
Country                199
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna(subset = ["Country"], inplace=True)

/Users/jennyliang/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
preferred_cities_df.count()

City_ID                199
City                   199
Lat                    199
Lng                    199
Max Temp               199
Current Description    199
Humidity               199
Cloudiness             199
Wind Speed             199
Country                199
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Miandrivazo,MG,76.28,-19.5167,45.4667,
4,Kapaa,US,75.52,22.0752,-159.3190,
6,Roebourne,AU,81.52,-20.7833,117.1333,
10,Odweyne,SO,76.24,9.4092,45.0640,
13,Arraial Do Cabo,BR,76.96,-22.9661,-42.0278,
19,Kieta,PG,77.00,-6.2167,155.6333,
26,Cabo San Lucas,MX,82.51,22.8909,-109.9124,
30,Kirakira,SB,78.31,-10.4544,161.9205,
38,Meulaboh,ID,76.73,4.1363,96.1285,
41,Saint-Philippe,RE,76.59,-21.3585,55.7679,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Miandrivazo,MG,76.28,-19.5167,45.4667,Hôtel Arc En Ciel Miandrivazo
4,Kapaa,US,75.52,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
6,Roebourne,AU,81.52,-20.7833,117.1333,Latitude20 Roebourne Village
10,Odweyne,SO,76.24,9.4092,45.0640,Eng Hiirey House
13,Arraial Do Cabo,BR,76.96,-22.9661,-42.0278,Pousada Porto Praia
...,...,...,...,...,...,...
664,Hobyo,SO,80.19,5.3505,48.5268,Halane Hotel and Restaurent
670,Paracuru,BR,87.82,-3.4100,-39.0306,Hotel Vento Brasil
674,Cap-Haitien,HT,83.25,19.7578,-72.2042,Habitation Jouissant
680,Rio Claro,BR,82.22,-22.4114,-47.5614,Hotel Vivendas Rio Claro by Atlantica


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)

In [12]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Call the figure to plot the data.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))